In [ ]:
from __init__ import *

In [ ]:
directory_path = "{}copy_number/".format(SETTING["directory_path"])

kraft.path.make(directory_path)

In [ ]:
for url in ("https://ndownloader.figshare.com/files/22629107",):

    kraft.internet.download(
        url, directory_path, name="CCLE_gene_cn.csv", overwrite=SETTING["overwrite"]
    )

In [ ]:
gene_x_sample = pd.read_csv("{}CCLE_gene_cn.csv".format(directory_path), index_col=0).T

gene_x_sample.index = (label.split()[0] for label in gene_x_sample.index.to_numpy())

gene_x_sample.columns = kraft.name_biology.name_cell_line(
    gene_x_sample.columns.to_numpy()
)

gene_x_sample.index.name = "Gene"

gene_x_sample.to_csv(
    "{}gene_x_sample.tsv".format(directory_path),
    sep="\t",
)

gene_x_sample

In [ ]:
mean = np.nanmean(gene_x_sample.to_numpy())

std = np.nanstd(gene_x_sample.to_numpy())

loss = mean - std * 3

gain = mean + std * 3

print(loss, gain)


def make_gene_loss_gain_x_(sample_copy_number_, loss, gain):

    gene_loss_gain_x_ = np.full((2, sample_copy_number_.size), 0)

    for sample_index, copy_number in enumerate(sample_copy_number_.to_numpy()):

        if copy_number <= loss:

            gene_loss_gain_x_[0, sample_index] = 1

        elif gain <= copy_number:

            gene_loss_gain_x_[1, sample_index] = 1

    has_1_ = (gene_loss_gain_x_ == 1).any(axis=1)

    gene = sample_copy_number_.name

    return pd.DataFrame(
        data=gene_loss_gain_x_[has_1_],
        index=np.asarray(("{} loss".format(gene), "{} gain".format(gene)))[has_1_],
        columns=sample_copy_number_.index,
    )

In [ ]:
path = "{}gene_loss_gain_x_sample.tsv".format(directory_path)

write_header = True

with open(path, mode="w") as io:

    for _, sample_copy_number_ in gene_x_sample.iterrows():

        make_gene_loss_gain_x_(sample_copy_number_, loss, gain).to_csv(
            io, sep="\t", header=write_header, mode="a"
        )

        if write_header:

            write_header = False

gene_loss_gain_x_sample = pd.read_csv(path, sep="\t", index_col=0)

gene_loss_gain_x_sample